In [1]:
pip install python-dotenv groq requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00


In [2]:
# from dotenv import load_dotenv
import os
from groq import Groq
import requests

# # Load environment variables from .env file
# load_dotenv()


In [3]:
from google.colab import userdata
os.environ["GROQ_API_KEY"]=userdata.get('GROQ_API_KEY')

In [4]:
class Tool:
    def __init__(self, name, function):
        self.name = name
        self.function = function

    def execute(self, *args, **kwargs):
        return self.function(*args, **kwargs)


In [5]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        self.tools = {}
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def add_tool(self, tool: Tool):
        self.tools[tool.name] = tool

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        response = self.execute()
        if response.startswith("CALL_TOOL"):
            parts = response.split()
            tool_name = parts[1]
            params = parts[2:]
            result = self.tools[tool_name].execute(*params)
            self.messages.append({"role": "tool", "content": result})
            return result
        else:
            self.messages.append({"role": "assistant", "content": response})
            return response

    def execute(self):
        completion = self.client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content


In [6]:
def calculator(a, b, operation):
    a = float(a)
    b = float(b)
    if operation == "add":
        return str(a + b)
    elif operation == "subtract":
        return str(a - b)
    elif operation == "multiply":
        return str(a * b)
    elif operation == "divide":
        return str(a / b)
    else:
        return "Invalid operation"

calc_tool = Tool("calculator", calculator)


In [7]:
def web_search(query):
    response = requests.get(f"https://api.duckduckgo.com/?q={query}&format=json&pretty=1")
    if response.status_code == 200:
        return response.json()["results"]
    else:
        return "Failed to fetch results"

search_tool = Tool("web_search", web_search)


In [10]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
client = Groq()
agent = Agent(client, system="You are a helpful assistant.")

# Add tools to the agent
agent.add_tool(calc_tool)
agent.add_tool(search_tool)

# Call the web search tool
response = agent(" what is weather today in new york")
print(response)


I'd be happy to help!

According to current weather reports, here's the weather forecast for New York City today:

**Current Weather:**

* Conditions: Partly Cloudy
* Temperature: 64°F (18°C)
* Humidity: 64%
* Wind: NW 10 mph (16 km/h)

**Forecast for the Day:**

* High Temperature: 68°F (20°C)
* Low Temperature: 56°F (13°C)
* Precipitation: 0% chance of rain
* UV Index: 6 (Moderate)

**Additional Details:**

* Sunrise: 6:28 AM EDT
* Sunset: 7:31 PM EDT

Please note that weather forecasts are subject to change, and it's always a good idea to check for updates throughout the day.

If you'd like more detailed information or a forecast for a specific area within New York City, feel free to let me know!
